In [1]:
import os
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
from google.colab import files
from PIL import ImageOps


In [2]:
def main():
    output_directory = "rus_kursiv_alphabet_images"
    os.makedirs(output_directory, exist_ok=True)

    alphabet = 'А Б В Г Д Е Ё Ж З И Й К Л М Н О П Р С Т У Ф Х Ц Ч Ш Щ Ъ Ы Ь Э Ю Я'.split()

    font_path = "Arial Italic.ttf"

    font_size = 100

    font = ImageFont.truetype(font_path, font_size)

    features_data = []
    header = ["Character", "Top Left Mass", "Top Right Mass", "Bottom Left Mass", "Bottom Right Mass",
              "Normalized Top Left Mass", "Normalized Top Right Mass", "Normalized Bottom Left Mass",
              "Normalized Bottom Right Mass", "Center of Gravity X", "Center of Gravity Y",
              "Normalized COG X", "Normalized COG Y", "Horizontal Moment of Inertia",
              "Vertical Moment of Inertia", "Normalized Horizontal Moment of Inertia",
              "Normalized Vertical Moment of Inertia"]



    for char in alphabet:
        image_size = (200, 200)  #
        image = Image.new("L", image_size, color=255)
        draw = ImageDraw.Draw(image)

        text_width, text_height = draw.textsize(char, font=font)

        draw.text(((image_size[0] - text_width) / 2, (image_size[1] - text_height) / 2), char, fill=0, font=font)


        inverted_image = ImageOps.invert(image)
        bbox = inverted_image.getbbox()
        if bbox:
            cropped_image = image.crop(bbox)

            image_filename = os.path.join(output_directory, f"{char}.png")
            cropped_image.save(image_filename)

            np_image = np.array(cropped_image)

            height, width = np_image.shape
            quarter_width = width // 2
            quarter_height = height // 2

            top_left = np_image[:quarter_height, :quarter_width]
            top_left_mass = np.sum(top_left == 0)

            top_right = np_image[:quarter_height, quarter_width:]
            top_right_mass = np.sum(top_right == 0)

            bottom_left = np_image[quarter_height:, :quarter_width]
            bottom_left_mass = np.sum(bottom_left == 0)

            bottom_right = np_image[quarter_height:, quarter_width:]
            bottom_right_mass = np.sum(bottom_right == 0)

            quarter_area = quarter_width * quarter_height

            norm_top_left_mass = top_left_mass / quarter_area
            norm_top_right_mass = top_right_mass / quarter_area
            norm_bottom_left_mass = bottom_left_mass / quarter_area
            norm_bottom_right_mass = bottom_right_mass / quarter_area

            y_indices, x_indices = np.where(np_image == 0)
            cog_x = np.mean(x_indices)
            cog_y = np.mean(y_indices)

            norm_cog_x = cog_x / width
            norm_cog_y = cog_y / height

            h_moment_inertia = np.sum((y_indices - cog_y) ** 2)
            v_moment_inertia = np.sum((x_indices - cog_x) ** 2)

            norm_h_moment_inertia = h_moment_inertia / np.sum(np_image == 0)
            norm_v_moment_inertia = v_moment_inertia / np.sum(np_image == 0)

            features_data.append([
                char,
                top_left_mass,
                top_right_mass,
                bottom_left_mass,
                bottom_right_mass,
                norm_top_left_mass,
                norm_top_right_mass,
                norm_bottom_left_mass,
                norm_bottom_right_mass,
                cog_x,
                cog_y,
                norm_cog_x,
                norm_cog_y,
                h_moment_inertia,
                v_moment_inertia,
                norm_h_moment_inertia,
                norm_v_moment_inertia
            ])

            profile_x = np.sum(np_image == 0, axis=0)
            profile_y = np.sum(np_image == 0, axis=1)


            plt.figure()
            plt.bar(range(len(profile_x)), profile_x)
            plt.xlabel('Column')
            plt.ylabel('Black Pixel Count')
            plt.title(f'Profile X for Character {char}')
            plt.savefig(os.path.join(output_directory, f"{char}_profile_x.png"))
            plt.close()

            # Профиль Y
            plt.figure()
            plt.bar(range(len(profile_y)), profile_y)
            plt.xlabel('Row')
            plt.ylabel('Black Pixel Count')
            plt.title(f'Profile Y for Character {char}')
            plt.savefig(os.path.join(output_directory, f"{char}_profile_y.png"))
            plt.close()



    csv_filename = os.path.join(output_directory, "features.csv")
    with open(csv_filename, mode='w', newline='') as csv_file:
        writer = csv.writer(csv_file, delimiter=';')
        writer.writerow(header)
        writer.writerows(features_data)

    print(f"Изображения символов, профили и CSV-файл сохранены в директории: {output_directory}")


    files.download(csv_filename)


if __name__ == "__main__":
    main()

<ipython-input-2-e7dda8106341>:28: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(char, font=font)
<ipython-input-2-e7dda8106341>:28: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(char, font=font)
<ipython-input-2-e7dda8106341>:28: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(char, font=font)
<ipython-input-2-e7dda8106341>:28: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(char, font=font)
<ipython-input-2-e7dda8106341>:28: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-

Изображения символов, профили и CSV-файл сохранены в директории: rus_kursiv_alphabet_images


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>